### 기업 정보 수집: 회사명, 섹터, 시가총액, PER, PBR, 배당수익률 등
### 재무제표 수집:
   - 손익계산서 (Income Statement)
   - 재무상태표 (Balance Sheet)
   - 현금흐름표 (Cash Flow Statement)
### 배당금 정보 수집
### 데이터 요약 및 파일 저장

### 수집 대상 기업:
    -- AAPL (Apple)
    -- MSFT (Microsoft)
    -- TSLA (Tesla)
    -- GOOGL (Alphabet)
    -- AMZN (Amazon)

In [1]:
# yfinance 재무제표 및 기업정보 수집
import os
import pandas as pd
import yfinance as yf
from datetime import datetime

os.makedirs('data/collected', exist_ok=True)

# 분석할 기업 티커 목록
tickers = ['AAPL', 'MSFT', 'TSLA', 'GOOGL', 'AMZN']


In [ ]:
# 기업 정보 수집
company_info = []
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # 주요 정보 추출
        company_data = {
            'ticker': ticker,
            'name': info.get('longName', ''),
            'sector': info.get('sector', ''),
            'industry': info.get('industry', ''),
            'market_cap': info.get('marketCap', 0),
            'pe_ratio': info.get('trailingPE', 0),
            'pb_ratio': info.get('priceToBook', 0),
            'dividend_yield': info.get('dividendYield', 0),
            'revenue': info.get('totalRevenue', 0),
            'employees': info.get('fullTimeEmployees', 0),
            'country': info.get('country', ''),
            'website': info.get('website', '')
        }
        company_info.append(company_data)
        print(f"{ticker} 기업 정보 수집 완료")
    except Exception as e:
        print(f"{ticker} 기업 정보 수집 오류: {e}")
        continue

company_df = pd.DataFrame(company_info)


In [ ]:
# 손익계산서 (Income Statement) 수집
income_statements = []
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        income_stmt = stock.financials
        
        if income_stmt is not None and not income_stmt.empty:
            # 데이터 변환
            income_stmt = income_stmt.T.reset_index()
            income_stmt['ticker'] = ticker
            income_stmt = income_stmt.rename(columns={'index': 'date'})
            income_statements.append(income_stmt)
            print(f"{ticker} 손익계산서 수집 완료")
    except Exception as e:
        print(f"{ticker} 손익계산서 수집 오류: {e}")
        continue

income_df = pd.concat(income_statements, ignore_index=True) if income_statements else pd.DataFrame()


In [ ]:
# 재무상태표 (Balance Sheet) 수집
balance_sheets = []
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        balance_sheet = stock.balance_sheet
        
        if balance_sheet is not None and not balance_sheet.empty:
            balance_sheet = balance_sheet.T.reset_index()
            balance_sheet['ticker'] = ticker
            balance_sheet = balance_sheet.rename(columns={'index': 'date'})
            balance_sheets.append(balance_sheet)
            print(f"{ticker} 재무상태표 수집 완료")
    except Exception as e:
        print(f"{ticker} 재무상태표 수집 오류: {e}")
        continue

balance_df = pd.concat(balance_sheets, ignore_index=True) if balance_sheets else pd.DataFrame()


In [ ]:
# 현금흐름표 (Cash Flow Statement) 수집
cash_flows = []
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        cash_flow = stock.cashflow
        
        if cash_flow is not None and not cash_flow.empty:
            cash_flow = cash_flow.T.reset_index()
            cash_flow['ticker'] = ticker
            cash_flow = cash_flow.rename(columns={'index': 'date'})
            cash_flows.append(cash_flow)
            print(f"{ticker} 현금흐름표 수집 완료")
    except Exception as e:
        print(f"{ticker} 현금흐름표 수집 오류: {e}")
        continue

cash_flow_df = pd.concat(cash_flows, ignore_index=True) if cash_flows else pd.DataFrame()


In [ ]:
# 배당금 정보 수집
dividend_data = []
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        dividends = stock.dividends
        
        if dividends is not None and not dividends.empty:
            dividends = dividends.reset_index()
            dividends['ticker'] = ticker
            dividends = dividends.rename(columns={'index': 'date', 'Dividends': 'dividend_amount'})
            dividend_data.append(dividends)
            print(f"{ticker} 배당금 정보 수집 완료")
    except Exception as e:
        print(f"{ticker} 배당금 정보 수집 오류: {e}")
        continue

dividend_df = pd.concat(dividend_data, ignore_index=True) if dividend_data else pd.DataFrame()


In [ ]:
# 주요 지표 요약
print("\n=== 기업 정보 요약 ===")
if not company_df.empty:
    print(company_df[['ticker', 'name', 'sector', 'market_cap', 'pe_ratio', 'pb_ratio']])

print("\n=== 손익계산서 데이터 형태 ===")
if not income_df.empty:
    print(f"수집된 손익계산서 항목 수: {len(income_df.columns)}")
    print(f"기간: {income_df['date'].min()} ~ {income_df['date'].max()}")

print("\n=== 배당금 지급 현황 ===")
if not dividend_df.empty:
    print(dividend_df.groupby('ticker').size().reset_index(name='배당지급횟수'))


In [ ]:
# 파일로 저장
company_df.to_csv('data/collected/yfinance_company_info.csv', index=False, encoding='utf-8-sig')
if not income_df.empty:
    income_df.to_csv('data/collected/yfinance_income_statement.csv', index=False, encoding='utf-8-sig')
if not balance_df.empty:
    balance_df.to_csv('data/collected/yfinance_balance_sheet.csv', index=False, encoding='utf-8-sig')
if not cash_flow_df.empty:
    cash_flow_df.to_csv('data/collected/yfinance_cash_flow.csv', index=False, encoding='utf-8-sig')
if not dividend_df.empty:
    dividend_df.to_csv('data/collected/yfinance_dividends.csv', index=False, encoding='utf-8-sig')

print("\n=== 파일 저장 완료 ===")
print("- 기업 정보: data/collected/yfinance_company_info.csv")
print("- 손익계산서: data/collected/yfinance_income_statement.csv")
print("- 재무상태표: data/collected/yfinance_balance_sheet.csv")
print("- 현금흐름표: data/collected/yfinance_cash_flow.csv")
print("- 배당금 정보: data/collected/yfinance_dividends.csv")


### data/collected/ 폴더에 다음과 같은 CSV 파일들이 생성
 - yfinance_company_info.csv - 기업 기본 정보
 - yfinance_income_statement.csv - 손익계산서
 - yfinance_balance_sheet.csv - 재무상태표
 - yfinance_cash_flow.csv - 현금흐름표
 - yfinance_dividends.csv - 배당금 정보